In [8]:
from flyanalysis import video
import os
import glob
import tqdm
import csv

In [7]:
root_folder = "/home/buchsbaum/mnt/md0/Videos/"
exp_folder = "20230519_130210"

video_files = glob.glob(os.path.join(root_folder, exp_folder, "*.mp4"))
print(f"Detected {len(video_files)} video files")

Detected 450 video files


In [ ]:
for vf in tqdm(video_files):
    output_filename = vf.replace(".mp4", ".csv")
    with open(output_filename, "w") as f:
        writer = csv.writer(f)
        writer.writerow(["x", "y", "z", "xvel", "yvel", "zvel"])
        for i, frame in enumerate(video.read_video(vf)):
            gray = video.gray(frame)
            thresh = video.threshold(gray)
            contours = video.find_contours(thresh)
            contour = video.get_largest_contour(contours)
            contour_params = video.get_contour_parameters(contour)
